In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [3]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* 元データ（予測環境の生データ）
* Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ

In [4]:
# 元データによる結果を取得する
# ✅1. Extra-Pに読み込ますデータの取得
# 2. 取得できた数式から予測


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName = "cg",
    conditions={
        "process":input_list_process,
        "na":cg_input_list_na,
        "nonzer":cg_input_list_nonzer,
        "niter":cg_input_list_niter,
        "shift":cg_input_list_shift
    }
)

filePath = "./extra-p_docker/share/input_cg.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


/root/src/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/root/src/lib/lab_lib.py:8781: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [5]:
%sx pwd
%sx which extrap
%ls
%sx extrap --text /workspace/extra-p_docker/share/input.txt

Dockerfile*    csv_files/           root_jupyter/       usedNote2.ipynb*
Pipfile*       docker-compose.yml*  testing_pri.ipynb*  usedNote3.ipynb*
Pipfile.lock*  docs/                testing_pub.ipynb*  usedNote4.ipynb*
README.md*     extra-p_docker/      txt_files/          used_scripts/
app/           lib/                 usedNote.ipynb*


['',
 'Loading file: |          | [00:00<?]ERROR: The given file path is not valid.',
 '',
 'Loading file: |          | [00:00<?]']

In [6]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

str_ExtraPinputData :str = gen_ExtraPinputData(
    benchmarkName="lulesh",
    conditions = {
        "process" : train_lulesh_processes,
        "iterations" : train_lulesh_iterations,
        "sizes" : train_lulesh_sizes
    },
    version=2
)

filePath = "./extra-p_docker/share/input_lulesh.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)


KeyError: 'Inclusive'

In [ ]:
rawDF_lulesh :pd.DataFrame = return_rawDFinLULESH (
    processes=train_lulesh_processes,
    iterations=train_lulesh_iterations,
    sizes=train_lulesh_sizes,
    csvDirPath=csvDirPath
)

rawDF_lulesh

In [ ]:
def return_rawDF_cg(
    list_process :list[int],
    list_iteration :list[int],
    list_size: list[int],
    csvDir :str
):
    """ベンチマークプログラムLULESHのプロファイルを取得する関数
    
    Args:
        list_process(list[int]):プロセス数のリスト
        list_iteration(list[int]):初期変数iterationのリスト
        list_size(list[int]):初期変数sizeのリスト
        csvDir(str):CSVファイルを格納したディレクトリのパスを表す文字列
        
    Returns:
        pd.DataFrame
        
    """
    
    list_before_concat_DF: list[pd.DataFrame] = []
    for elem_process in list_process:
        for elem_iteration in list_iteration:
            for elem_size in list_size:
                filePath :str = f"{csvDir}pprof_lulesh_p{elem_process}i{elem_iteration}s{elem_size}.csv"
                if os.path.isfile(filePath):
                    try:
                        DF_read_raw: pd.DataFrame = pd.read_csv(filePath)
                        DF_read_raw["process"] = elem_process
                        DF_read_raw["iteration"] = elem_iteration
                        DF_read_raw["size"] = elem_size
                        list_before_concat_DF.appen(DF_read_raw)
                    except:
                        warnings.warn(f"{filePath} is empty.")
                else:
                    warnings.warn(f"{filePath} doesn't exist")
    print(list_before_concat_DF)
    return pd.concat(objs=list_before_concat_DF, axis=0)

return_rawDF_cg(
    list_process = test_lulesh_processes,
    list_iteration = test_lulesh_iterations,
    list_size = test_lulesh_sizes,
    csvDir=csvDirPath
)


In [ ]:
%reset

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___